In [6]:
# --- SILVER: normalise + restrict to South Africa test matches (1992+) ---

# Setup (run this once after starting/restarting the kernel)
from pathlib import Path
import pandas as pd
import numpy as np

bronze_path = Path("data/bronze/bronze_results.parquet")  # adjust if different
MIN_YEAR = 1992
TEAM_NAME = "South Africa"


def normalise_columns(df: pd.DataFrame) -> pd.DataFrame:
    lower = {c.lower().strip(): c for c in df.columns}
    ren = {}

    # canonical names
    for k in ["date", "match_date"]:
        if k in lower: ren[lower[k]] = "date"; break
    for k in ["home_team", "team1", "team_a"]:
        if k in lower: ren[lower[k]] = "team1"; break
    for k in ["away_team", "team2", "team_b", "opponent"]:
        if k in lower: ren[lower[k]] = "team2"; break
    for k in ["home_score", "score1", "points_for", "home_points"]:
        if k in lower: ren[lower[k]] = "score1"; break
    for k in ["away_score", "score2", "points_against", "away_points"]:
        if k in lower: ren[lower[k]] = "score2"; break
    for k in ["neutral", "is_neutral", "neutral_venue"]:
        if k in lower: ren[lower[k]] = "neutral"; break
    for k in ["tournament", "competition"]:
        if k in lower: ren[lower[k]] = "tournament"; break
    for k in ["venue", "city", "stadium"]:
        if k in lower: ren[lower[k]] = "venue"; break
    for k in ["country"]:
        if k in lower: ren[lower[k]] = "country"; break

    df = df.rename(columns=ren)

    if "date" not in df:
        raise ValueError("No date column found.")
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    for s in ["score1", "score2"]:
        if s in df:
            df[s] = pd.to_numeric(df[s], errors="coerce")

    # ensure optional columns exist
    for c in ["tournament", "venue", "country"]:
        if c not in df.columns:
            df[c] = pd.NA

    # normalise neutral flag
    if "neutral" in df.columns:
        if df["neutral"].dtype == object:
            df["neutral"] = df["neutral"].astype(str).str.lower().isin(["true", "1", "yes"])
        else:
            df["neutral"] = df["neutral"].astype("boolean")
    else:
        df["neutral"] = pd.NA

    return df

# read Bronze and build Silver
df_bronze = pd.read_parquet(bronze_path)
df_bronze = normalise_columns(df_bronze)

df_bronze["year"] = df_bronze["date"].dt.year

# South Africa matches from 1992 onwards
df_sa = df_bronze[
    (df_bronze["year"] >= MIN_YEAR) &
    ((df_bronze["team1"] == TEAM_NAME) | (df_bronze["team2"] == TEAM_NAME))
].copy()

# SA perspective
sa_is_t1 = (df_sa["team1"] == TEAM_NAME)
df_sa["sa_team"]   = TEAM_NAME
df_sa["opp_team"]  = np.where(sa_is_t1, df_sa["team2"], df_sa["team1"])
df_sa["sa_score"]  = np.where(sa_is_t1, df_sa["score1"], df_sa["score2"])
df_sa["opp_score"] = np.where(sa_is_t1, df_sa["score2"], df_sa["score1"])

# home flag: team1 is home unless neutral == True
df_sa["home"] = np.select(
    [
        df_sa["neutral"] == True,
        sa_is_t1,
        ~sa_is_t1
    ],
    [0, 1, 0],
    default=pd.NA
)

# outcomes
df_sa["score_margin"] = df_sa["sa_score"] - df_sa["opp_score"]
df_sa["win"] = (df_sa["score_margin"] > 0).astype(int)

# keep essential columns
keep = [
    "date", "year", "sa_team", "opp_team",
    "sa_score", "opp_score", "score_margin", "win",
    "home", "tournament", "venue", "country", "neutral"
]
df_silver = df_sa[keep].sort_values("date").reset_index(drop=True)

silver_path = Path("data/silver/silver_results.parquet")
df_silver.to_parquet(silver_path, index=False)
print(f"[SILVER] Saved {silver_path} (rows={len(df_silver)})")


[SILVER] Saved data\silver\silver_results.parquet (rows=328)
